In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from bokeh.plotting import figure
from bokeh.io import  push_notebook,output_notebook, show
from bokeh.models.widgets import Panel, Tabs #Bokeh 2.x
# from bokeh.models import TabPanel, Tabs  #Bokeh 3.x
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.io import curdoc

pd.options.display.max_columns = 60
pd.options.display.max_rows = 50

#### Table des matieres

1. [**Import Dataset**](#1.-Import-Dataset)
2. [**Supression des erreures**](#2.-Supression-des-erreurs)
    1.  [Dataset Vehicules](#2.1-Dataset-Vehicules)
    2.  [Dataset Caracteristiques](#2.2-Dataset-Caracteristiques)
    3.  [Dataset Usager](#2.3-Dataset-Usager)
    4.  [Dataset Lieux](#2.4-Dataset-Lieux)
3. [**Supression des colonnes(variables)**](#3.-Supression-des-colonnes(variables))
4. [**Gestion des Nan**](#4.-Gestion-des-Nan)
    1.  [Remplacement -1 par Nan](#4.-Gestion-des-Nan)
    2.  [Dataset Vehicules](#4.2-Dataset-Vehicules)
    3.  [Dataset Caracteristiques](#4.3-Dataset-Caracteristiques)
    4.  [Dataset Usager](#4.4-Dataset-Usager)
    5.  [Dataset Lieux](#4.5-Dataset-Lieux)
5. [**Creation de variables suppl. et regroupement de categories**](#5.-Creation-de-variables-suppl.-et-regroupement-de-categories)
    1.  [Dataset Vehicules](#5.1-Dataset-Vehicules)
    2.  [Dataset Caracteristiques](#5.2-Dataset-Caracteristiques)
    3.  [Dataset Usager](#5.3-Dataset-Usager)
    4.  [Dataset Lieux](#5.4-Dataset-Lieux)
6. [**Gestion des doublons**](#6.-Gestion-des-doublons)
7. [**Sauvegarde dataset finale**](#7.-Sauvegarde-dataset-finale)

### 1. Import Dataset

In [53]:

path = '../data/'

######
years = ['2018f', '2019', '2020',  '2021' ]
s_df = ["carcteristiques","lieux-","vehicules-","usagers-"]
dff = pd.DataFrame()
k=0
i=0
hows='inner'
######
for year in years:
    df_1 =   pd.read_csv(path + "caracteristiques-" + year + '.csv', delimiter=';')
    df_2  =  pd.read_csv(path + "lieux-" + year + '.csv', delimiter=';')
    df_3  =  pd.read_csv(path + "vehicules-" + year + '.csv', delimiter=';')
    df_4 =   pd.read_csv(path + "usagers-" + year + '.csv', delimiter=';')

    merged_df = pd.merge(df_1,      df_2,on=['Num_Acc'], how=hows)
    merged_df = pd.merge(merged_df, df_3,on=['Num_Acc'], how=hows)
    if year=='2018f':
        df = pd.merge(merged_df, df_4,on=['Num_Acc','num_veh'], how=hows)
    else:
        df = pd.merge(merged_df, df_4,on=['Num_Acc','num_veh','id_vehicule'], how=hows)

    dff = pd.concat([dff, df],ignore_index=True)
    k=k+df.shape[0]
    #print('year : ', year,df.shape)
    i=i+1   
print("Total dataset 2018-2021 : ",dff.shape)

C:\Users\dufou\AppData\Local\Temp\ipykernel_23844\1905508614.py:13: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2  =  pd.read_csv(path + "lieux-" + year + '.csv', delimiter=';')


Total dataset 2018-2021 :  (497594, 57)


In [54]:
col_veh   = ['Num_Acc','senc','catv','occutc','obs','obsm','choc', 'manv', 'num_veh', 'id_vehicule', 'motor']
col_carac = ['Num_Acc','an','mois','jour','hrmn','lum','agg','int','atm','col','com','adr','gps','lat','long','dep']
col_lieu  = ['Num_Acc','catr','voie','v1','v2','circ','nbv','pr','pr1','vosp','prof','plan','lartpc','larrout','surf',
             'infra','situ','env1','vma']
col_usag  = ['Num_Acc','place','catu','grav','sexe','trajet','secu','locp','actp','etatp','an_nais','num_veh',
             'id_vehicule','secu1','secu2','secu3']

#df_veh   = pd.DataFrame();  df_veh   = dff[col_veh]
#df_carac = pd.DataFrame();  df_carac = dff[col_carac]
#df_lieu  = pd.DataFrame();  df_lieu  = dff[col_lieu]
#df_usag  = pd.DataFrame();  df_usag  = dff[col_usag]

### 2. Supression des erreurs

#### 2.1 Dataset Vehicules 

In [55]:
# Dans la variable id_vehicule il faut remplacer '\xa' par '' : 138\xa0306 -> 1380306
dff['id_vehicule'] = dff['id_vehicule'].str.replace('\xa0', '')

## Dans la variable num_veh, il faut supprimer les erreurs de frappe '\01' et ']01'
dff = dff.drop(dff[dff['num_veh'].isin(['\01', ']01'])].index)

#### 2.2 Dataset Caracteristiques

In [56]:
# Colonne 'an' : transformation de l'année 18 en 2018
dff['an'] = dff['an'].apply(lambda x: 2018 if x == 18 else x)
# Colonne 'hrmn' : Standardisation de la colonne. cause : format 2019,2020 et 2021 = str avec séparateur ":" des heures et des minutes tandis que format 2018 = numérique sans ":"
dff["hrmn"] = dff["hrmn"].astype(str)
dff['hrmn'] = dff['hrmn'].str.zfill(4)
dff.loc[dff['an']==2018, 'hrmn'] = dff['hrmn'].apply(lambda x: x[:2] + ":" + x[2:])
dff['hrmn'][dff['an']==2018].sort_values().head(10)
# Colonne 'dep' (département) : double format : string pour 2019,2020 et 2021 et numérique pour 2018 -> objectif les mettre au format numérique pour faciliter les travaux ultérieurs
    # Corse département 2A et 2B -> 20
dff['dep'] = dff['dep'].replace(['2A', '2B'], '20')
    # Transformation de la colonne dep de type str à int
dff['dep'] = dff['dep'].astype(int)
    # Les numéros des départements métropolitain du dataset 2018 doivent être mis au bon format
dff['dep'] = dff['dep'].apply(lambda x: 200 if x in [201, 202] else x)
dff["dep"] = dff.apply(lambda x: x["dep"] / 10 if x["an"] == 2018 and x["dep"] < 970 else x["dep"], axis=1)
dff['dep'] = dff['dep'].astype(int)
# Colonnes lat et long : 
# transformation de la ',' en '.'
dff['lat'] = dff['lat'].astype(str).str.replace(',', '.').astype(float)
dff['long'] = dff['long'].astype(str).str.replace(',', '.').astype(float)
dff["lat"] = pd.to_numeric(dff["lat"])
dff["long"] = pd.to_numeric(dff["long"])

# ajout de du point pour les valeurs de l'année 2018
dff["lat"] = dff.apply(lambda x: x["lat"] / 100000 if x["an"] == 2018 else x["lat"], axis=1)
dff["long"] = dff.apply(lambda x: x["long"] / 100000 if x["an"] == 2018 else x["long"], axis=1)

# problème de la polarité des latitudes pour 2018 : Elles sont toutes positives alors qu'elles devraient être négatives pour les départements 988, 976, 974, 987, 986 et 972
departements = ['988', '976', '974', '987', '986', '972']
def fonction_lat(ligne):
    if ligne['an'] == 2018 and ligne['dep'] in departements:
        return ligne['lat'] * -1
    else:
        return ligne['lat']

dff['lat'] = dff.apply(fonction_lat, axis=1)
# problème de la longitude pour 2018. Absence de longitude négative = irrésolu. De toute façon la colonne sera supprimée pour notre modélisation


# Concaténation les colonnes 'an', 'mois' et 'jour' en une seule chaîne
df['date'] = df['an'].astype(str) + '-' + df['mois'].astype(str) + '-' + df['jour'].astype(str)

# Conversion la colonne 'date' en dates à l'aide de pd.to_datetime()
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

#### 2.3 Dataset Usager

In [57]:
# variable actp action piéton
# présence de int, float et char 
# toutes les valeurs sont remplacées par des int 

# B = inconnu  
dff['actp'] = dff['actp'].replace([0, 0.0], 0) \
                         .replace([1, 1.0], 1) \
                         .replace([2, 2.0], 2) \
                         .replace([3, 3.0], 3) \
                         .replace([4, 4.0], 4) \
                         .replace([5, 5.0], 5) \
                         .replace([6, 6.0], 6) \
                         .replace([7, 7.0], 7) \
                         .replace([8, 8.0], 8) \
                         .replace([9, 9.0], 9) \
                         .replace(['A'],   10) \
                         .replace(['B'],   -1) \
                         .replace([' -1'], -1)   

#  changement du type variable objet -> int
dff['actp'] = dff['actp'].replace(np.NaN, -1)
dff['actp'] = dff['actp'].astype(np.int64)
# print(dff['actp'].dtype)'

In [58]:
# Fusion des variables secu et secu1
ind= dff.index[(dff['secu'] == 11.0)].tolist()
dff.loc[ind, 'secu1'] = 1.0
ind= dff.index[(dff['secu'] == 21.0)].tolist()
dff.loc[ind, 'secu1'] = 2.0
ind= dff.index[(dff['secu'] == 31.0)].tolist()
dff.loc[ind, 'secu1'] = 3.0
ind= dff.index[(dff['secu'] == 41.0)].tolist()
dff.loc[ind, 'secu1'] = 4.0
ind= dff.index[(dff['secu'] == 91.0)].tolist()
dff.loc[ind, 'secu1'] = 9.0

#######
#à verifier 
#mod = {
#    11.0: 1.0,
#    21.0: 2.0,
#    31.0: 3.0,
#    41.0: 4.0,
#    91.0: 9.0
#}

#dff['secu1'] = dff['secu'].replace(mod, inplace=False)
####

In [59]:
# Remplacement des valeurs "inconnue" ou "non renseigné" par NaN

# remplacement dans catu des 4 par des Nan 
dff['catu'] = dff['catu'].replace(4, np.nan)

# Remplacement 0 par Nan
columns_replace = ['etatp', 'trajet', 'secu2', 'secu3', 'locp', 'actp']
dff[columns_replace] = dff[columns_replace].replace(0, np.nan)

##########
#OLD
# Remplacement 0 par Nan
#dff['etatp'] = dff['etatp'].replace(0, np.nan)
#dff['trajet'] = dff['trajet'].replace(0, np.nan)
#dff['secu2'] = dff['secu2'].replace(0, np.nan)
#dff['secu3'] = dff['secu3'].replace(0, np.nan)
#dff['locp'] = dff['locp'].replace(0, np.nan)
#dff['actp'] = dff['actp'].replace(0, np.nan)
##########

#### 2.4 Dataset Lieux 

In [60]:
# Variable circ, prof, plan
# présence de 0 non prévue dans la documentation
# remplacement par NaN

dff['circ'] = dff['circ'].replace(0, np.nan)
dff['prof'] = dff['prof'].replace(0, np.nan)
dff['plan'] = dff['plan'].replace(0, np.nan)

### 3. Supression des colonnes(variables)

In [61]:
print ('Avant supression ',dff.shape)
##### Vehicules #####
#Suppression des colonnes non pertinentes
# 'num_veh' 'Num_Acc' 'id_vehicule' seront suprimmés apres creation nouvelles variables  
dff = dff.drop(columns=['motor','occutc','senc']) 

##### Caracteristiques #####
liste = ['com', 'adr', 'gps']
dff = dff.drop(liste, axis=1)

##### Usager #####
# suppression des variables avec > 90% NaN
liste_usagers = ['secu', 'secu2', 'secu3', 'locp', 'etatp', 'actp']
dff = dff.drop(liste_usagers, axis=1)

##### Lieux #####
# Suppression des colonnes de mauvaise qualité ou non pertinentes (pr, pr1)
liste_lieux = ['voie', 'v1', 'v2', 'pr', 'pr1', 'lartpc', 'larrout', 'env1', 'vma']
dff.drop(liste_lieux, axis=1, inplace=True)

print ('Apres supression ',dff.shape)

Avant supression  (497593, 57)
Apres supression  (497593, 36)


### 4. Gestion des Nan
#### 4.1 Remplacement -1 par Nan

In [62]:
#print(dff.isna().sum())
dff = dff.replace(-1, np.nan)
#print(dff.isna().sum())

In [63]:
# Suppression des lignes où la variable cible vaut NaN

# print(dff['grav'].value_counts(dropna=False))
ind= dff.index[(dff['grav'].isna())].tolist()
dff=dff.drop(dff.index[ind])
dff.shape

(497533, 36)

#### 4.2 Dataset Vehicules 

In [64]:
# Vérification du pourcentage de valeurs manquantes dans les variables choisies
g_nan_veh = ['Num_Acc','catv','obs','obsm','choc', 'manv', 'num_veh']
round((dff[g_nan_veh].isna().sum()/dff[g_nan_veh].shape[0]).sort_values(ascending=False)*100,2)
dff=dff.dropna(subset=g_nan_veh)

In [61]:
#print(dff.shape)
#round((dff[g_nan_veh].isna().sum()/dff[g_nan_veh].shape[0]).sort_values(ascending=False)*100,2)

#### 4.3 Dataset Caracteristiques

In [65]:
#Vérification du pourcentage de valeurs manquantes dans les variables choisies

g_nan_car = ['int','lum','atm','mois','col', 'hrmn']
round((dff[g_nan_car].isna().sum()/dff[g_nan_car].shape[0]).sort_values(ascending=False)*100,2)
dff=dff.dropna(subset=g_nan_car)

In [63]:
#print(dff.shape)
#round((dff[g_nan_car].isna().sum()/dff[g_nan_car].shape[0]).sort_values(ascending=False)*100,2)

#### 4.4 Dataset Usager

In [55]:
#OLD version
# Traitement des NaN dans trajet

# print(dff["trajet"].value_counts(dropna= False))
# 25% NaN => remplacement par valeur modale 5 Promenade – loisir
#dff["trajet"]= dff["trajet"].fillna(5)
#print(dff["trajet"].value_counts(dropna= False))

# Traitement des NaN dans place

# print(dff["place"].value_counts(dropna= False))
# 2% NaN => suppression des lignes 
#dff=dff.dropna(subset=["place"])
#print(dff["place"].value_counts(dropna= False))

# Traitement des NaN dans sexe

# print(dff["sexe"].value_counts(dropna= False))
#  0.06%% NaN => suppression des lignes
#dff=dff.dropna(subset=["sexe"])
#print(dff["sexe"].value_counts(dropna= False))

# Traitement des NaN dans secu1

# print(dff["secu1"].value_counts(dropna= False))
#  5% NaN => suppression des lignes
#dff=dff.dropna(subset=["secu1"])
#print(dff["secu1"].value_counts(dropna= False))


# Traitement des NaN dans an_nais

# print(dff["an_nais"].value_counts(dropna= False))
# NaN => suppresion des lignes
#dff=dff.dropna(subset=["an_nais"])
#print(dff["an_nais"].value_counts(dropna= False))


In [66]:
g_nan_usager = ['Num_Acc', 'num_veh','an_nais','place','catu','sexe','trajet','secu1', 'grav']

g_nan_usager_supr_lignes = ['place','sexe','secu1','an_nais']

# Traitement des NaN dans trajet
# 25% NaN => remplacement par valeur modale 5 Promenade – loisir

dff["trajet"]= dff["trajet"].fillna(5)
#print(dff["trajet"].value_counts(dropna= False))

#place
# 2% NaN => suppression des lignes 
#sexe
# 0.06% NaN => suppression des lignes
#secu1
# 5% NaN => suppression des lignes
#an_nais
# NaN => suppresion des lignes

dff=dff.dropna(subset = g_nan_usager_supr_lignes )

In [67]:
# Vérification du pourcentage de valeurs manquantes dans les variables choisies
round((dff[g_nan_usager].isna().sum()/dff[g_nan_usager].shape[0]).sort_values(ascending=False)*100,2)

Num_Acc    0.0
num_veh    0.0
an_nais    0.0
place      0.0
catu       0.0
sexe       0.0
trajet     0.0
secu1      0.0
grav       0.0
dtype: float64

#### 4.5 Dataset Lieux 

In [58]:
# OLD cersion
# Traitement des NaN dans circ
# 5% NaN => suppresion des lignes
#dff=dff.dropna(subset=["circ"])
# print(dff["circ"].value_counts(dropna= False))

# Traitement des NaN dans nbv
# 1% NaN => suppresion des lignes
#dff=dff.dropna(subset=["nbv"])
# print(dff["nbv"].value_counts(dropna= False))

# Traitement des NaN dans infra
# 1% NaN => suppresion des lignes
#dff=dff.dropna(subset=["infra"])
# print(dff["infra"].value_counts(dropna= False))

# Traitement des NaN dans vosp
# 1% NaN => suppresion des lignes
#dff=dff.dropna(subset=["vosp"])
# print(dff["vosp"].value_counts(dropna= False))

# Traitement des NaN dans prof
# 1% NaN => suppresion des lignes
#dff=dff.dropna(subset=["prof"])
# print(dff["prof"].value_counts(dropna= False))

# Traitement des NaN dans situ
# 1% NaN => suppresion des lignes
#dff=dff.dropna(subset=["situ"])
# print(dff["situ"].value_counts(dropna= False))

# Traitement des NaN dans surf
# 1% NaN => suppresion des lignes
#dff=dff.dropna(subset=["surf"])
# print(dff["surf"].value_counts(dropna= False))

# Traitement des NaN dans plan
# 1% NaN => suppresion des lignes
#dff=dff.dropna(subset=["plan"])
# print(dff["plan"].value_counts(dropna= False))

In [68]:
#  circ  - 5% NaN => suppresion des lignes
#  nbv   - 1% NaN => suppresion des lignes
#  infra - 1% NaN => suppresion des lignes
#  vosp  - 1% NaN => suppresion des lignes
#  prof  - 1% NaN => suppresion des lignes
#  situ  - 1% NaN => suppresion des lignes
#  surf   - 1% NaN => suppresion des lignes
#  plan   - 1% NaN => suppresion des lignes

g_nan_Lieux_supr_lignes = ['circ','nbv','infra','vosp','prof','situ','surf','plan']

dff=dff.dropna(subset = g_nan_Lieux_supr_lignes )

In [69]:
# Vérification du pourcentage de valeurs manquantes dans les variables choisies
g_nan_lieux = ['catr','circ','nbv','vosp','prof','plan','surf','infra', 'situ']
#round((dff[g_nan_lieux].isna().sum()/dff[g_nan_lieux].shape[0]).sort_values(ascending=False)*100,2)

### 5. Creation de variables suppl. et regroupement de categories

#### 5.1 Dataset Vehicules 

In [70]:
# Nouvelle variable : quantité de voitures dans chaque accident pour chaque usager
'''
nb_veh = dff.groupby('Num_Acc')['num_veh'].nunique().reset_index()
nb_veh = nb_veh.rename(columns={'num_veh': 'nb_veh'})
dff = dff.merge(nb_veh, on='Num_Acc', how='left')
'''

mask = dff['an'] == 2018

nb_veh_masked   = dff[mask] .groupby('Num_Acc')['num_veh'].    nunique().reset_index()
nb_veh_unmasked = dff[~mask].groupby('Num_Acc')['id_vehicule'].nunique().reset_index()

nb_veh_masked =   nb_veh_masked.rename(columns={'num_veh': 'nb_veh'})
nb_veh_unmasked = nb_veh_unmasked.rename(columns={'id_vehicule': 'nb_veh'})

nb_veh_fus = pd.concat([nb_veh_masked, nb_veh_unmasked])
dff = dff.merge(nb_veh_fus, on='Num_Acc', how='left')

dff.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,lat,long,dep,catr,circ,nbv,vosp,prof,plan,surf,infra,situ,catv,obs,obsm,choc,manv,num_veh,place,catu,grav,sexe,trajet,an_nais,id_vehicule,secu1,nb_veh
0,201800000001,2018,1,24,15:05,1.0,1,4.0,1.0,1.0,50.55737,2.94992,59,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,7.0,0.0,2.0,3.0,1.0,B01,1.0,1,3.0,1.0,5.0,1928.0,NaN,1.0,2
1,201800000001,2018,1,24,15:05,1.0,1,4.0,1.0,1.0,50.55737,2.94992,59,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,7.0,0.0,2.0,2.0,15.0,A01,1.0,1,1.0,1.0,5.0,1960.0,NaN,1.0,2
2,201800000002,2018,2,12,10:15,1.0,2,7.0,7.0,7.0,50.52936,2.93151,59,4,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,7.0,0.0,1.0,0.0,1.0,A01,1.0,1,1.0,1.0,5.0,1947.0,NaN,1.0,1
3,201800000003,2018,3,4,11:35,1.0,2,3.0,1.0,7.0,50.51243,2.91714,59,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,33.0,1.0,2.0,1.0,1.0,A01,1.0,1,3.0,1.0,5.0,1987.0,NaN,2.0,1
4,201800000004,2018,5,5,17:35,1.0,2,1.0,7.0,3.0,50.51974,2.89123,59,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,7.0,0.0,9.0,2.0,21.0,A01,1.0,1,1.0,1.0,5.0,1982.0,NaN,1.0,1


In [71]:
#cheking
#pd.options.display.max_rows = 100
#dff[dff['nb_veh'] >10].head(100)

**Regroupement de categories**

In [71]:
#Variable choc
#dff['choc'].value_counts()

dff['choc'] = dff['choc'].replace([1.0, 2.0, 3.0],1).replace([4.0, 5.0, 6.0],2).replace([7.0, 8.0],3).replace([9.0],4)
dff['choc'].value_counts()

choc
1.0    277416
2.0     72511
3.0     53961
0.0     20629
4.0      7233
Name: count, dtype: int64

In [72]:
#Variable manv
#dff['catv'].value_counts()
dff['catv'] = dff['catv'].replace([1, 2, 4, 5, 6, 30, 35, 41, 50, 60, 80], 2) \
                         .replace([3, 7, 8, 9, 10], 1) \
                         .replace([31, 32, 33, 34, 36, 42, 43], 3) \
                         .replace([11, 12, 13, 14, 15, 16, 17, 20, 21], 4) \
                         .replace([18, 19, 37, 38, 39, 40], 5) \
                         .replace([99], 6)
dff['catv'].value_counts()

catv
1.0    308223
3.0     55205
2.0     48036
4.0     12466
5.0      5705
6.0      1680
0.0       435
Name: count, dtype: int64

#### 5.2 Dataset Caracteristiques

Création des variables - date, heure, week end

In [73]:
jours = { 0: 'Lundi', 1: 'Mardi', 2: 'Mercredi', 3: 'Jeudi', 4: 'Vendredi', 5: 'Samedi', 6: 'Dimanche'}

# création de la colonne date
    # Concaténation les colonnes 'an', 'mois' et 'jour' en une seule chaîne
dff['date'] = dff['an'].astype(str) + '-' + dff['mois'].astype(str) + '-' + dff['jour'].astype(str)
    # Conversion la colonne 'date' en dates à l'aide de pd.to_datetime()
dff['date'] = pd.to_datetime(dff['date'], format='%Y-%m-%d')

# Création de la colonne week day
dff["weekday"] = dff['date'].dt.dayofweek.map(jours)

# Création de la colonne weekend
dff["weekend"] = np.where(dff["weekday"].isin(['Samedi', 'Dimanche']), 1, 0)

# Création de la colonne Heure
    #  'hrmn' -> objet datetime
dff['datetime'] = pd.to_datetime(dff['hrmn'], format='%H:%M')
    # Extraction heure et les minutes sous forme de nombre à virgule flottante
dff['heure_decimal'] = dff['datetime'].dt.hour + dff['datetime'].dt.minute/60
    # Arrondi de l'heure à l'heure la plus proche
dff['heure'] = dff['heure_decimal'].round().astype(int)
dff.loc[dff['heure'] == 24, 'heure'] = 0

# Création de la colonne période de la journée
def periode_jour(heure):
    if 6 <= heure < 12:
        return 'Matin'
    elif 12 <= heure < 18:
        return 'Après-midi'
    elif 18 <= heure < 24:
        return 'Soir'
    else:
        return 'Nuit'

dff['periode'] = dff['heure'].apply(periode_jour)


Supression des variables (2)

In [65]:
# liste = ['hrmn', 'an', 'jour', 'weekday', 'heure_decimal', 'datetime', 'date', 'lat', 'long']
# liste = ['hrmn', 'jour', 'weekday', 'heure_decimal', 'datetime', 'date', 'lat', 'long']
# dff = dff.drop(liste, axis=1)

#SIPRIMMER AN a la fin

Catégorisation des variables - int (intersection), lum (lumière), col (collision), atm(atmosphère)

In [74]:
# Variable int (intersection)
    # création de la catégorie 3 : "Intersection et autre"
    # Création de la catégorie 2 "Passage à niveau"
dff['int'] = dff['int'].replace([2, 4, 5, 6, 7, 9], 3).replace(8, 2)

# Variable lum (lumière)
    # création de la catégorie 1 : "Conditions lumineuse optimale"
    # Maintien de la catégorie 2 "Crépuscule au aube"
    # Création de la catégorie 3 "Nuit sans éclairage"
    
dff['lum'] = dff['lum'].replace([1, 5], 1).replace([3, 4], 3)

# Variable col (collision)
    # création de la catégorie 4 : "Autres collisions entre véhicules"
    # Création de la catégorie 3 "deux véhicules frontales"
    # Création de la catégorie 2 "Autres collisions"
    # Création de la catégorie 2 "Sans collisions"
dff['col'] = dff['col'].replace([2, 3, 4, 5], 4).replace(1, 3).replace(6, 2).replace(7, 1)

# Variable atm (atmosphère)
    # création de la catégorie 1 : "Normale"
    # Création de la catégorie 2 "Pluie"
    # Création de la catégorie 3 "Neige-grêle"
    # Création de la catégorie 4 "Brouillard-fumée"
    # Création de la catégorie 5 "Vent fort - tempête"

dff['atm'] = dff['atm'].replace([1, 7, 8, 9], 1).replace([2, 3],2).replace(4, 3).replace(5, 4).replace(6,5)


#### 5.3 Dataset Usager

In [75]:
# Age de usager
dff["age"] = dff["an"]- dff["an_nais"]
##
age_sexe_cond1 = (
    dff[(dff['catu'] == 1) & (dff['an'] == 2018)]
    .groupby(['Num_Acc', 'num_veh'])
    .agg({'age': 'unique', 'sexe': 'unique'})   #'unique' -> 'min'
    .reset_index()
)

age_sexe_cond2 = (
    dff[(dff['catu'] == 1) & (dff['an'] >= 2019)]
    .groupby(['Num_Acc', 'id_vehicule'])
    .agg({'age': 'unique', 'sexe': 'unique'})
    .reset_index()
)
#age_sexe_cond1['age'] = age_sexe_cond1['age'].apply(lambda x: pd.to_numeric(pd.Series(x).unique()[0], errors='coerce'))
#age_sexe_cond2['age'] = age_sexe_cond2['age'].apply(lambda x: pd.to_numeric(pd.Series(x).unique()[0], errors='coerce'))
#age_sexe_cond1['sexe'] = age_sexe_cond1['sexe'].apply(lambda x: pd.to_numeric(pd.Series(x).unique()[0], errors='coerce'))
#age_sexe_cond2['sexe'] = age_sexe_cond2['sexe'].apply(lambda x: pd.to_numeric(pd.Series(x).unique()[0], errors='coerce'))

age_sexe_cond1[['age', 'sexe']] = age_sexe_cond1[['age', 'sexe']].applymap(lambda x: pd.to_numeric(pd.Series(x).unique()[0], errors='coerce'))
age_sexe_cond2[['age', 'sexe']] = age_sexe_cond2[['age', 'sexe']].applymap(lambda x: pd.to_numeric(pd.Series(x).unique()[0], errors='coerce'))

age_sexe_cond1 = age_sexe_cond1.rename(columns={'age': 'age_cond','sexe': 'sexe_cond'})
age_sexe_cond2 = age_sexe_cond2.rename(columns={'age': 'age_cond','sexe': 'sexe_cond'})

data_merged1 = dff.merge(age_sexe_cond1, on=['Num_Acc', 'num_veh'])
data_merged2 = dff.merge(age_sexe_cond2, on=['Num_Acc', 'id_vehicule'])
dff_merged = pd.concat([data_merged1, data_merged2])

dff_merged.head()
print('Taille de dataset avant transformation', dff.shape)
print('Taille de dataset apres transformation',dff_merged.shape)
#print(dff_merged.isnull().sum())

C:\Users\dufou\AppData\Local\Temp\ipykernel_23844\2388853541.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_sexe_cond1[['age', 'sexe']] = age_sexe_cond1[['age', 'sexe']].applymap(lambda x: pd.to_numeric(pd.Series(x).unique()[0], errors='coerce'))
C:\Users\dufou\AppData\Local\Temp\ipykernel_23844\2388853541.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  age_sexe_cond2[['age', 'sexe']] = age_sexe_cond2[['age', 'sexe']].applymap(lambda x: pd.to_numeric(pd.Series(x).unique()[0], errors='coerce'))


Taille de dataset avant transformation (431750, 45)
Taille de dataset apres transformation (427777, 47)


In [97]:
#pd.options.display.max_rows = 150
#dff_merged['age'].value_counts()

In [76]:
lab_age_usag = ['0-2', '3-6','7-10', '11-17','18-24', '25-64', '65-74','75-plus']
bins_age_usg = [-1, 3, 7, 11, 18, 25, 65, 75, np.inf]
lab_age_cond = ['0-7','8-13','14-17','18-24', '25-34','35-44','45-54', '55-64', '65-74','75-94', '95-plus']
bins_age_con = [-1, 8, 14, 18, 25, 35, 45, 55, 65,75,95, np.inf]
	
dff_merged['age_usag']=pd.cut(x = dff_merged['age'],
                        bins = bins_age_usg, 
                        labels = lab_age_usag)



dff_merged['age_cond']=pd.cut(x = dff_merged['age_cond'],
                        bins = bins_age_con, 
                        labels = lab_age_cond)


print("Répartition (%) nombre usagers pour chaque tranche d'âge")
print(round(dff_merged['age_usag'].value_counts(normalize=True, sort=True)*100,2))
print ('------')
print("Répartition (%) nombre conducteurs pour chaque tranche d'âge")
print(round(dff_merged['age_cond'].value_counts(normalize=True, sort=True, ascending=False)*100,2))

dff_merged.head()

Répartition (%) nombre usagers pour chaque tranche d'âge
age_usag
25-64      60.55
18-24      19.16
11-17       7.09
65-74       5.59
75-plus     4.03
7-10        1.41
3-6         1.21
0-2         0.97
Name: proportion, dtype: float64
------
Répartition (%) nombre conducteurs pour chaque tranche d'âge
age_cond
25-34      22.92
18-24      19.84
35-44      18.00
45-54      15.36
55-64      10.14
65-74       5.60
14-17       4.09
75-94       3.52
8-13        0.42
95-plus     0.06
0-7         0.05
Name: proportion, dtype: float64


,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,lat,long,dep,catr,circ,nbv,vosp,prof,plan,surf,infra,situ,catv,obs,obsm,choc,manv,num_veh,place,catu,grav,sexe,trajet,an_nais,id_vehicule,secu1,nb_veh,date,weekday,weekend,datetime,heure_decimal,heure,periode,age,age_cond,sexe_cond,age_usag
0,201800000001,2018,1,24,15:05,1.0,1,3.0,1.0,3.0,50.55737,2.94992,59,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,B01,1.0,1,3.0,1.0,5.0,1928.0,NaN,1.0,2,2018-01-24,Mercredi,0,1900-01-01 15:05:00,15.083333,15,Après-midi,90.0,75-94,1.0,75-plus
1,201800000001,2018,1,24,15:05,1.0,1,3.0,1.0,3.0,50.55737,2.94992,59,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,15.0,A01,1.0,1,1.0,1.0,5.0,1960.0,NaN,1.0,2,2018-01-24,Mercredi,0,1900-01-01 15:05:00,15.083333,15,Après-midi,58.0,55-64,1.0,25-64
2,201800000002,2018,2,12,10:15,1.0,2,3.0,1.0,1.0,50.52936,2.93151,59,4,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,A01,1.0,1,1.0,1.0,5.0,1947.0,NaN,1.0,1,2018-02-12,Lundi,0,1900-01-01 10:15:00,10.250000,10,Matin,71.0,65-74,1.0,65-74
3,201800000003,2018,3,4,11:35,1.0,2,3.0,1.0,1.0,50.51243,2.91714,59,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,1.0,2.0,1.0,1.0,A01,1.0,1,3.0,1.0,5.0,1987.0,NaN,2.0,1,2018-03-04,Dimanche,1,1900-01-01 11:35:00,11.583333,12,Après-midi,31.0,25-34,1.0,25-64
4,201800000004,2018,5,5,17:35,1.0,2,1.0,1.0,4.0,50.51974,2.89123,59,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,9.0,1.0,21.0,A01,1.0,1,1.0,1.0,5.0,1982.0,NaN,1.0,1,2018-05-05,Samedi,1,1900-01-01 17:35:00,17.583333,18,Soir,36.0,35-44,1.0,25-64


In [77]:
#print(dff.shape)
#print(dff_merged.shape)

dff=dff_merged
#print(dff.shape)

# RENOMMAGE variable sexe en sexe_usagers
dff = dff.rename(columns={ 'sexe': 'sexe_usag' })

In [78]:
# regroupement catégories

# Variable place (place du conducteur)
    # Pas de modification catégorie 1 : place du conducteur
    # Pas de modification catégorie 2 : place avant droite 
    # Modification de la catégorie 3 "Autres" qui intègre les autres catégories

dff['place'] = dff['place'].replace([4, 5, 6, 7, 8, 9, 10], 3)
#dff['place'].value_counts()

In [79]:
 # regroupement catégories

# Variable trajet (motif du déplacement lors accident)
    # Pas de modification catégorie 1 : "Domicile-travail"
    # Modification de la catégorie 2 :  "Promenade – loisirs" (ancienne catégorie 5)
    # Modification de la catégorie 3 "Autres" qui intègre les autres catégories (domicile - école, courses - achats, utilisation pofessionnelle)

dff['trajet'] = dff['trajet'].replace(2,9)
dff['trajet'] = dff['trajet'].replace([4, 6, 7, 8, 9], 3)
dff['trajet'] = dff['trajet'].replace(5,2)
#dff['trajet'].value_counts(normalize = True)

In [80]:
 # regroupement catégories

# Variable secu1 (1er équipement de sécurité)
# les valeurs 1, 2 et 8 sont très corrélées avec la valeur Tuée de la variable cible

    # Pas de modification catégorie 1 : "Ceinture"
    # Modification de la catégorie 2 :  "Casque"
    # Modification de la catégorie 3 "Non déterminable" (ancienne catégorie 8)
    # Modification de la catégorie 4 "Autres" qui contient les autres catégories (disposiitif enfants, gilet réfléchissant, airbag, gants, airbag + gants)

dff['secu1'] = dff['secu1'].replace([0, 3, 5, 6, 7, 9], 4)
dff['secu1'] = dff['secu1'].replace(8,3)
#dff['secu1'].value_counts(normalize = True)

#### 5.4 Dataset Lieux 

In [81]:
 # regroupement catégories

# Variable catr = Catégorie de route
    # Pas de modification catégorie 1 : "Autoroute"
    # Pas de modification de la catégorie 2 : "Route nationale"
    # Pas de modification de la catégorie 3 " "Route Départementale"
    # Pas de modification de la catégorie 4 : "Voie Communales"
    # Regroupement des catégories 5 à 9 en catégorie 5 "Autre"  (nb pas de catégorie 8)

dff['catr'].replace([6, 7, 9], 5, inplace=True)
#dff['catr'].value_counts(normalize = True)

In [82]:
 # regroupement catégories

# Variable nbv = Nombre total de voies de circulation
     # Il y a du zéro voie, ce qui peut être un parking par exemple.
     # Au dela de 8 voies, la représentation est quasi-inexistante, 
     # Regroupement des 9 voies et plus en 9  

dff['nbv'].replace([10, 11, 12, 13], 9, inplace=True)
#dff['nbv'].value_counts(normalize = True)

In [83]:
 # regroupement catégories

# Variable  surf = Etat de la surface
    # Pas de modification catégorie 1 : "Normale"
    # Pas de modification de la catégorie 2 : "Mouillée"
    # Regroupement des catégories 3 à 9 en catégorie 3 "Autre" 

dff['surf'].replace([0, 4, 5, 6, 7, 8, 9], 3, inplace=True)
#dff['surf'].value_counts(normalize = True)

### 6. Gestion des doublons

In [84]:
dff.head()

,Num_Acc,an,mois,jour,hrmn,lum,agg,int,atm,col,lat,long,dep,catr,circ,nbv,vosp,prof,plan,surf,infra,situ,catv,obs,obsm,choc,manv,num_veh,place,catu,grav,sexe_usag,trajet,an_nais,id_vehicule,secu1,nb_veh,date,weekday,weekend,datetime,heure_decimal,heure,periode,age,age_cond,sexe_cond,age_usag
0,201800000001,2018,1,24,15:05,1.0,1,3.0,1.0,3.0,50.55737,2.94992,59,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,B01,1.0,1,3.0,1.0,2.0,1928.0,NaN,1.0,2,2018-01-24,Mercredi,0,1900-01-01 15:05:00,15.083333,15,Après-midi,90.0,75-94,1.0,75-plus
1,201800000001,2018,1,24,15:05,1.0,1,3.0,1.0,3.0,50.55737,2.94992,59,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,15.0,A01,1.0,1,1.0,1.0,2.0,1960.0,NaN,1.0,2,2018-01-24,Mercredi,0,1900-01-01 15:05:00,15.083333,15,Après-midi,58.0,55-64,1.0,25-64
2,201800000002,2018,2,12,10:15,1.0,2,3.0,1.0,1.0,50.52936,2.93151,59,4,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,A01,1.0,1,1.0,1.0,2.0,1947.0,NaN,1.0,1,2018-02-12,Lundi,0,1900-01-01 10:15:00,10.250000,10,Matin,71.0,65-74,1.0,65-74
3,201800000003,2018,3,4,11:35,1.0,2,3.0,1.0,1.0,50.51243,2.91714,59,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,1.0,2.0,1.0,1.0,A01,1.0,1,3.0,1.0,2.0,1987.0,NaN,2.0,1,2018-03-04,Dimanche,1,1900-01-01 11:35:00,11.583333,12,Après-midi,31.0,25-34,1.0,25-64
4,201800000004,2018,5,5,17:35,1.0,2,1.0,1.0,4.0,50.51974,2.89123,59,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,9.0,1.0,21.0,A01,1.0,1,1.0,1.0,2.0,1982.0,NaN,1.0,1,2018-05-05,Samedi,1,1900-01-01 17:35:00,17.583333,18,Soir,36.0,35-44,1.0,25-64


In [85]:
### Suppression finale des variables qui ne sont pas utiles après les transformation
dff = dff.drop(columns=['num_veh', 'age','an_nais','id_vehicule']) 
dff = dff.set_index('Num_Acc')

In [86]:
print('Nombre de lignes avant', dff.shape[0])
#print("Nombre de doublons :", dff.duplicated().sum())
#doublons = dff.duplicated(keep=False)
#doublons_index = dff.index[doublons]
#print(doublons_index)
#doublons = dff.loc[doublons_index]
dff = dff.drop_duplicates()
print('Nombre de lignes apres', dff.shape[0])

Nombre de lignes avant 427777
Nombre de lignes apres 423783


In [87]:
dff.head()
#dff.shape

,an,mois,jour,hrmn,lum,agg,int,atm,col,lat,long,dep,catr,circ,nbv,vosp,prof,plan,surf,infra,situ,catv,obs,obsm,choc,manv,place,catu,grav,sexe_usag,trajet,secu1,nb_veh,date,weekday,weekend,datetime,heure_decimal,heure,periode,age_cond,sexe_cond,age_usag
Num_Acc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201800000001,2018,1,24,15:05,1.0,1,3.0,1.0,3.0,50.55737,2.94992,59,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1,3.0,1.0,2.0,1.0,2,2018-01-24,Mercredi,0,1900-01-01 15:05:00,15.083333,15,Après-midi,75-94,1.0,75-plus
201800000001,2018,1,24,15:05,1.0,1,3.0,1.0,3.0,50.55737,2.94992,59,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,15.0,1.0,1,1.0,1.0,2.0,1.0,2,2018-01-24,Mercredi,0,1900-01-01 15:05:00,15.083333,15,Après-midi,55-64,1.0,25-64
201800000002,2018,2,12,10:15,1.0,2,3.0,1.0,1.0,50.52936,2.93151,59,4,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1,1.0,1.0,2.0,1.0,1,2018-02-12,Lundi,0,1900-01-01 10:15:00,10.250000,10,Matin,65-74,1.0,65-74
201800000003,2018,3,4,11:35,1.0,2,3.0,1.0,1.0,50.51243,2.91714,59,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1,3.0,1.0,2.0,2.0,1,2018-03-04,Dimanche,1,1900-01-01 11:35:00,11.583333,12,Après-midi,25-34,1.0,25-64
201800000004,2018,5,5,17:35,1.0,2,1.0,1.0,4.0,50.51974,2.89123,59,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,9.0,1.0,21.0,1.0,1,1.0,1.0,2.0,1.0,1,2018-05-05,Samedi,1,1900-01-01 17:35:00,17.583333,18,Soir,35-44,1.0,25-64


In [88]:
dff = dff[dff['an'] != 2018]

In [89]:
dff.dropna(inplace=True)

In [90]:
dff.isna().sum()

an               0
mois             0
jour             0
hrmn             0
lum              0
agg              0
int              0
atm              0
col              0
lat              0
long             0
dep              0
catr             0
circ             0
nbv              0
vosp             0
prof             0
plan             0
surf             0
infra            0
situ             0
catv             0
obs              0
obsm             0
choc             0
manv             0
place            0
catu             0
grav             0
sexe_usag        0
trajet           0
secu1            0
nb_veh           0
date             0
weekday          0
weekend          0
datetime         0
heure_decimal    0
heure            0
periode          0
age_cond         0
sexe_cond        0
age_usag         0
dtype: int64

### 7. Sauvegarde dataset finale

In [91]:
dff.to_csv('merged_data_2019_2021_for_streamlit.csv', index=True)

In [92]:
dff.isna().sum().sum()

0

In [93]:
dff.info()

<class 'pandas.core.frame.DataFrame'>
Index: 328550 entries, 201900000001 to 202100056518
Data columns (total 43 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   an             328550 non-null  int64         
 1   mois           328550 non-null  int64         
 2   jour           328550 non-null  int64         
 3   hrmn           328550 non-null  object        
 4   lum            328550 non-null  float64       
 5   agg            328550 non-null  int64         
 6   int            328550 non-null  float64       
 7   atm            328550 non-null  float64       
 8   col            328550 non-null  float64       
 9   lat            328550 non-null  float64       
 10  long           328550 non-null  float64       
 11  dep            328550 non-null  int32         
 12  catr           328550 non-null  int64         
 13  circ           328550 non-null  float64       
 14  nbv            328550 non-null  float64 

In [94]:
dff.dep.unique().shape

(106,)

In [102]:
dep_gps = pd.read_csv('c:/Users/dufou/OneDrive/Z-Python/Datascientest/Projet/FEV23_CDS_accidents/Exploratory Data Analysis/dep_gps.csv', sep=';')

In [103]:
dep_gps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   dep       106 non-null    int64  
 1   nom       106 non-null    object 
 2   Aire      106 non-null    object 
 3   Commune   106 non-null    object 
 4   dep_long  106 non-null    float64
 5   dep_lat   106 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 5.1+ KB


In [104]:
merged_df = dff.merge(dep_gps, on='dep', how='left')

In [105]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328550 entries, 0 to 328549
Data columns (total 48 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   an             328550 non-null  int64         
 1   mois           328550 non-null  int64         
 2   jour           328550 non-null  int64         
 3   hrmn           328550 non-null  object        
 4   lum            328550 non-null  float64       
 5   agg            328550 non-null  int64         
 6   int            328550 non-null  float64       
 7   atm            328550 non-null  float64       
 8   col            328550 non-null  float64       
 9   lat            328550 non-null  float64       
 10  long           328550 non-null  float64       
 11  dep            328550 non-null  int32         
 12  catr           328550 non-null  int64         
 13  circ           328550 non-null  float64       
 14  nbv            328550 non-null  float64       
 15  

In [106]:
merged_df.to_csv('merged_data_2019_2021_for_streamlit.csv', index=True)

In [48]:
result = merged_df.groupby(['dep', 'grav', 'dep_lat', 'dep_long']).size().reset_index(name='count_accidents')


In [49]:
result.head(95)

,dep,grav,dep_lat,dep_long,count_accidents
0,1,1.0,46.099444,5.348889,1001
1,1,2.0,46.099444,5.348889,108
2,1,3.0,46.099444,5.348889,644
3,1,4.0,46.099444,5.348889,1092
4,2,1.0,49.559444,3.558333,467
...,...,...,...,...,...
90,23,3.0,46.090278,2.018889,101
91,23,4.0,46.090278,2.018889,38
92,24,1.0,45.104167,0.741389,441
93,24,2.0,45.104167,0.741389,71


In [159]:
result[result['grav'] == 2].sort_values(by='count_accidents', ascending=False).head(50)

,dep,grav,dep_lat,dep_long,count_accidents
49,13,2.0,43.323600,5.051100,259
233,59,2.0,50.265000,3.131400,237
129,33,2.0,44.493100,-0.343100,199
329,83,2.0,43.273800,6.130500,182
133,34,2.0,43.344700,3.220200,182
173,44,2.0,47.214100,-1.405600,178
305,77,2.0,48.373600,2.560000,174
121,31,2.0,43.213100,1.102200,167
149,38,2.0,45.154800,5.343400,160
381,971,2.0,16.265000,-61.551000,157


In [50]:
map_df = result.groupby(['dep_long', 'dep_lat', 'grav']).size().reset_index(name='count')

In [51]:
map_df.head(10)

,dep_long,dep_lat,grav,count
0,-6.102281e+07,14.635541,1.0,1
1,-6.102281e+07,14.635541,2.0,1
2,-6.102281e+07,14.635541,3.0,1
3,-6.102281e+07,14.635541,4.0,1
4,-1.494260e+02,-17.650920,1.0,1
5,-1.494260e+02,-17.650920,2.0,1
6,-1.494260e+02,-17.650920,3.0,1
7,-1.494260e+02,-17.650920,4.0,1
8,-9.606524e+01,29.631349,1.0,1
9,-9.606524e+01,29.631349,2.0,1
